In [ ]:
!pip install scikit-learn==0.24

# Multiple Linear Regression
***

- Multiple linear regression differs from the simple one because it can handle multiple input features
- It is a simple algorithm initially developed in the field of statistics and was studied as a model for understanding the relationship between input and output variables
- It is a linear model - assumes linear relationship between input variables (X) and the output variable (y)
- Used to predict continious values (e.g., weight, price...)


#### Assumptions
1. **Linear Assumption** — model assumes the relationship between variables is linear
2. **No Noise** — model assumes that the input and output variables are not noisy — so remove outliers if possible
3. **No Collinearity** — model will overfit when you have highly correlated input variables
4. **Normal Distribution** — the model will make more reliable predictions if your input and output variables are normally distributed. If that’s not the case, try using some transforms on your variables to make them more normal-looking
5. **Rescaled Inputs** — use scalers or normalizer to make more reliable predictions

#### Take-home point
- Training a linear regression model means calculating the best coefficients for the line equation formula
- The best coefficients can be obtained with **gradient descent**
    - An iterative optimization algorithm that calculates derivatives wrt. each coefficient, and updates the coefficients on the go
    - One additional parameter - *learning rate*, specifies the rate at which coefficients are updated
        - High learning rate can lead to "missing" the best values
        - Low learning rate can lead to slow optimization

<br>

## Math behind
- In a nutshell, we need to calculate an array of weights and a single bias through gradient descent
- Weights = slopes, bias = y intercept
- We're still solving a simple line equation:

$$ \large \hat{y} = wx + b $$

<br>

**Multiple** linear regression.

<img src="https://miro.medium.com/max/414/1*Ko7YDmTa_TctiL2Fkm-kGQ.png">

<br>

- We also need a **cost function**
    - Function we want to minimize
    - Reducing the cost (loss) leads to better equation parameters
    - We'll choose Mean Squared Error (MSE), but feel free to work with any other

$$ \large MSE = \frac{1}{N} \sum_{i=1}^{n} ((y_i - \hat{y})^2 $$

<br>

- $ \hat{y} $ can be further written into:

$$ \large MSE = \frac{1}{N} \sum_{i=1}^{n} ((y_i - (wx_i + b))^2 $$

<br>

- To update weights and biases, we're using gradient descent
- It relies on partial derivatives calculation for each parameter
- Below you'll find derived MSE wrt. each parameter:

$$ \large \partial_w = \frac{1}{N} \sum_{i=1}^{n} 2x_i(\hat{y} - y) $$
<br>
$$ \large \partial_b = \frac{1}{N} \sum_{i=1}^{n} 2(\hat{y} - y) $$

<br>

- This $ 2 $ can be ommited, or you can leave it - it's not important
- Next, we're updating the existing weights and bias according to the following formulas:

$$ \large w = w - \alpha \cdot \partial_w $$
<br>
$$ \large b = b - \alpha \cdot \partial_b $$

- Where $ \alpha $ is the learning rate
- This process is then repeated for a predefined number of iterations
- Let's see this in action

<br>

## Implementation
- You'll need only Numpy to implement the logic
- Matplotlib is used for optional visualizations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (14, 7)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

<br>

- The `LinearRegression` class is written to follow the familiar Scikit-Learn syntax
- The coefficients are set to `None` at the start - `__init__()` method
- We're also keeping track of the loss (just for visualization purposes)
- The `fit()` method calculates the coefficients
- The `predict()` method essentially implements the line equation
- The `_mean_squared_error()` private function is used to calculate loss at every iteration

In [ ]:
class LinearRegression:
    '''
    A class which implements linear regression model with gradient descent.
    '''
    def __init__(self, learning_rate=0.01, n_iterations=10000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights, self.bias = None, None
        self.loss = []
        
    @staticmethod
    def _mean_squared_error(y, y_hat):
        '''
        Private method, used to evaluate loss at each iteration.
        
        :param: y - array, true values
        :param: y_hat - array, predicted values
        :return: float
        '''
        error = 0
        for i in range(len(y)):
            error += (y[i] - y_hat[i]) ** 2
        return error / len(y)
    
    def fit(self, X, y):
        '''
        Used to calculate the coefficient of the linear regression model.
        
        :param X: array, features
        :param y: array, true values
        :return: None
        '''
        # 1. Initialize weights and bias to zeros
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        
        # 2. Perform gradient descent
        for i in range(self.n_iterations):
            # Line equation
            y_hat = np.dot(X, self.weights) + self.bias
            loss = self._mean_squared_error(y, y_hat)
            self.loss.append(loss)
            
            # Calculate derivatives
            partial_w = (1 / X.shape[0]) * (2 * np.dot(X.T, (y_hat - y)))
            partial_d = (1 / X.shape[0]) * (2 * np.sum(y_hat - y))
            
            # Update the coefficients
            self.weights -= self.learning_rate * partial_w
            self.bias -= self.learning_rate * partial_d
        
        
    def predict(self, X):
        '''
        Makes predictions using the line equation.
        
        :param X: array, features
        :return: array, predictions
        '''
        return np.dot(X, self.weights) + self.bias

<br>

## Testing
- We'll use the *diabetes* dataset from Scikit-Learn. Data source: https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
X = data.data
y = data.target

- Let's look at the dataset:

In [ ]:
data_frame = load_diabetes(as_frame=True,  return_X_y=False)
data_frame

{'DESCR': '.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - age     age in years\n      - sex\n      - bmi     body mass index\n      - bp      average blood pressure\n      - s1      tc, T-Cells (a type of white blood cells)\n      - s2      ldl, low-density lipoproteins\n      - s3      hdl, high-density lipoproteins\n      - s4      tch, thyroid stimulating hormone\n      - s5      ltg, lamotrigine\n      - s6      glu, blood sugar l

- The below code cell applies train/test split to the dataset:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- You can now initialize and train the model, and afterwards make predictions:

In [ ]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1223: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


- These are the "optimal" weights:

In [ ]:
model.coef_

array([  56.18833323,  -41.39979092,  262.06089707,  187.45978128,
         28.69566099,    4.00266732, -144.32990897,  134.23708812,
        218.44767461,  132.3359013 ])

- And this is the "optimal" bias:

In [ ]:
model.intercept_

array([152.25970031])

<br>

## Loss Evaluation

- Let's visualize the loss at each iteration
- Ideally, we should see a curve that starts high and soon drops to zero or near zero:

In [ ]:
losses = {}
for lr in [0.5, 0.1, 0.01, 0.001]:
    model = SGDRegressor(max_iter=lr)
    model.fit(X_train, y_train)
    losses[f'LR={str(lr)}'] = model.loss

- You can now visualize loss at each iteration for these different learning rates:

- Seems like `learning_rate=0.5` works the best
- Let's train the model with it and evalute (print MSE):

In [ ]:
from sklearn.metrics import mean_squared_error
model = SGDRegressor(max_iter=1000, eta0=0.0005, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

mean_squared_error(y_test, preds)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1223: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


5021.854976744705



---


For Asynchronous Session (Sep 18):

1. Using the new diabetes dataset, what would be the optimum values of learning rate and epochs (# of iterations) to achieve the lowest possible MSE?
	
2. Following results from question 1, what are the values of the parameters (B1-B10) and B0 (intercept) you obtained that best represents the model for the disease progression?

3. Try applying multiple linear regression to a new dataset. Please find applicable datasets at [data.world](https://data.world/datasets/regression)
